In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.cluster import KMeans 
import plotly.graph_objs as go
import matplotlib.pyplot as plt
btc = yf.download(tickers = 'BTC-USD' , start = '2021-04-01', end = '2024-08-01')

# View the results
print(btc)

[*********************100%***********************]  1 of 1 completed
                    Open          High           Low         Close  \
Date                                                                 
2021-04-01  58926.562500  59586.070312  58505.277344  59095.808594   
2021-04-02  59098.878906  60267.187500  58869.281250  59384.312500   
2021-04-03  59397.410156  60110.269531  57603.890625  57603.890625   
2021-04-04  57604.839844  58913.746094  57168.675781  58758.554688   
2021-04-05  58760.875000  59891.296875  57694.824219  59057.878906   
...                  ...           ...           ...           ...   
2024-01-05  44192.980469  44353.285156  42784.718750  44162.691406   
2024-01-06  44178.953125  44227.632812  43475.156250  43989.195312   
2024-01-07  43998.464844  44495.570312  43662.230469  43943.097656   
2024-01-08  43948.707031  47218.000000  43244.082031  46970.503906   
2024-01-09  46987.640625  47127.710938  46322.882812  46756.839844   

               Adj C

In [2]:
# Convert adjusted closing price to numpy array
btc_prices = np.array(btc["Adj Close"])
print("BTC Prices:\n", btc_prices)

# Perform cluster analysis
K = 6
kmeans = KMeans(n_clusters=6).fit(btc_prices.reshape(-1, 1))

# predict which cluster each price is in
clusters = kmeans.predict(btc_prices.reshape(-1, 1))
print("Clusters:\n", clusters)


BTC Prices:
 [59095.80859375 59384.3125     57603.890625   ... 43943.09765625
 46970.50390625 46756.83984375]


C:\Users\User\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


Clusters:
 [1 1 1 ... 2 4 4]


In [3]:
# Assigns plotly as visualization engine
pd.options.plotting.backend = 'plotly'

# Arbitrarily 6 colors for our 6 clusters
colors = ['red', 'orange', 'yellow', 'green', 'blue', 'indigo']

# Create Scatter plot, assigning each point a color based
# on it's grouping where group_number == index of color.
fig = btc.plot.scatter(
    x=btc.index,
    y="Adj Close",
    color=[colors[i] for i in clusters],
)

# Configure some styles
layout = go.Layout(
    plot_bgcolor='#efefef',
    showlegend=False,
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=20,
    xaxis=dict(
        rangeslider=dict(
            visible=False
        ))
)
fig.update_layout(layout)

# Display plot in a local browser window
fig.show()

In [4]:
# Create list to hold values, initialized with infinite values
min_max_values = []

# init for each cluster group
for i in range(6):

    # Add values for which no price could be greater or less
    min_max_values.append([np.inf, -np.inf])

# Print initial values
print(min_max_values)

# Get min/max for each cluster
for i in range(len(btc_prices)):

    # Get cluster assigned to price
    cluster = clusters[i]

    # Compare for min value
    if btc_prices[i] < min_max_values[cluster][0]:
        min_max_values[cluster][0] = btc_prices[i]

    # Compare for max value
    if btc_prices[i] > min_max_values[cluster][1]:
        min_max_values[cluster][1] = btc_prices[i]
        
# Print resulting values
print(min_max_values)

[[inf, -inf], [inf, -inf], [inf, -inf], [inf, -inf], [inf, -inf], [inf, -inf]]
[[15787.2841796875, 24197.533203125], [54771.578125, 67566.828125], [39400.5859375, 45201.45703125], [24307.841796875, 32313.10546875], [45538.67578125, 54021.75390625], [32505.66015625, 39338.78515625]]


In [5]:
# Assign an arbitrary color to each of the 6 clusters
colors = ['red', 'orange', 'yellow', 'green', 'blue', 'indigo']

# Create Scatter plot, assigning each point a color where
# point group = color index.
fig = btc.plot.scatter(
    x=btc.index,
    y="Adj Close",
    color=[colors[i] for i in clusters],
)

# Add horizontal lines
for cluster_min, cluster_max in min_max_values:
    fig.add_hline(y=cluster_min, line_width=1, line_color="blue")
    fig.add_hline(y=cluster_max, line_width=1, line_color="blue")

# Add a trace of the price for better clarity
fig.add_trace(go.Trace(
    x=btc.index,
    y=btc['Adj Close'],
    line_color="black",
    line_width=1
))

# Make it pretty
layout = go.Layout(
    plot_bgcolor='#efefef',
    showlegend=False,
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=20,
    xaxis=dict(
        rangeslider=dict(
            visible=False
        ))
)
fig.update_layout(layout)
fig.show()

C:\Users\User\anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:687: DeprecationWarning:

plotly.graph_objs.Trace is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.




In [6]:
print("Initial Min/Max Values:\n", min_max_values)
# Create container for combined values
output = []

# Sort based on cluster minimum
s = sorted(min_max_values, key=lambda x: x[0])

# For each cluster get average of
for i, (_min, _max) in enumerate(s):

    # Append min from first cluster
    if i == 0:
        output.append(_min)

    # Append max from last cluster
    if i == len(min_max_values) - 1:
        output.append(_max)

    # Append average from cluster and adjacent for all others
    else:
        output.append(sum([_max, s[i+1][0]]) / 2)

print("Sorted Min/Max Values:\n", output)

Initial Min/Max Values:
 [[15787.2841796875, 24197.533203125], [54771.578125, 67566.828125], [39400.5859375, 45201.45703125], [24307.841796875, 32313.10546875], [45538.67578125, 54021.75390625], [32505.66015625, 39338.78515625]]
Sorted Min/Max Values:
 [15787.2841796875, 24252.6875, 32409.3828125, 39369.685546875, 45370.06640625, 54396.666015625, 67566.828125]


In [7]:
# Again, assign an arbitrary color to each of the 6 clusters
colors = ['red', 'orange', 'yellow', 'green', 'blue', 'indigo']

# Create Scatter plot, assigning each point a color where
# point group = color index.
fig = btc.plot.scatter(
    x=btc.index,
    y="Adj Close",
    color=[colors[i] for i in clusters],
)

# Add horizontal lines
for cluster_avg in output:
    fig.add_hline(y=cluster_avg, line_width=1, line_color="blue")




In [8]:
# Add a trace of the price for better clarity
fig.add_trace(go.Trace(
    x=btc.index,
    y=btc['Adj Close'],
    line_color="black",
    line_width=1
))

layout = go.Layout(
    plot_bgcolor='#efefef',
    showlegend=False,
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=20,
    xaxis=dict(
        rangeslider=dict(
            visible=False
        ))
)
fig.update_layout(layout)
fig.show()

C:\Users\User\anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:687: DeprecationWarning:

plotly.graph_objs.Trace is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.




In [9]:
# create a list to contain output values
values = []

# Define a range of cluster values to assess
K = range(1, 8)

# Performa a clustering using each value, save inertia_ value from each
for k in K:
    kmeans_n = KMeans(n_clusters=k)
    kmeans_n.fit(btc_prices.reshape(-1, 1))
    values.append(kmeans_n.inertia_)


C:\Users\User\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Users\User\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.

C:\Users\User\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

C:\Users\User\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.

C:\Users\User\an

In [10]:
# Create initial figure
fig = go.Figure()
# Add line plot of inertia values
fig.add_trace(go.Trace(
    x=list(K),
    y=values,
    line_color="black",
    line_width=1
))
# Make it pretty
layout = go.Layout(
    plot_bgcolor='#efefef',
    showlegend=False,
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=20,
    xaxis=dict(
        rangeslider=dict(
            visible=False
        ))
)
fig.update_layout(layout)
fig.show()


C:\Users\User\anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:687: DeprecationWarning:

plotly.graph_objs.Trace is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.


